## Importation de bibliothèques

In [66]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


## Création de la dataframe

In [67]:
df = pd.read_csv(r"After_Sort.csv", parse_dates=["BaseDateTime"])
df.sort_values("BaseDateTime", inplace=True)

## print informations

In [68]:
print(df.head())
print(df.info())
print(df.describe())
print(df["VesselType"].value_counts())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO CallSign  VesselType  Status  \
117352    206.0       MERCURY SKY     IMO9796949   9V5148          70       0   
117395    294.0  PROVINCETOWN III     IMO9329394  WDB8185          60       0   
117339    184.0         COBIA LNG     IMO9869306  9HA5192          84       0   
117360      NaN   BELLE CHASSE II            NaN  WDD7069          60       0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  WDK3453          60       0   

        Length  Width  Draft  Cargo TransceiverClass  
1

## ajout des colonnes lat et long +5, +10, +15 et delta_temps à la dataframe

In [69]:
nouvelles_colonnes = ['LONG_5', 'LAT_5', 'LONG_10', 'LAT_10', 'LONG_15', 'LAT_15','Delta_temps']

#ajout des colonnes valeurs vides(NaN)
for col in nouvelles_colonnes:
    df[col] = None 


print(df.head())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO  ... Draft  Cargo  \
117352    206.0       MERCURY SKY     IMO9796949  ...   NaN   70.0   
117395    294.0  PROVINCETOWN III     IMO9329394  ...   1.9   60.0   
117339    184.0         COBIA LNG     IMO9869306  ...   9.4   84.0   
117360      NaN   BELLE CHASSE II            NaN  ...   NaN   60.0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  ...   3.3   60.0   

        TransceiverClass  LONG_5  LAT_5  LONG_10  LAT_10 LONG_15 LAT_15  \
117352                 A    None   None     Non

## Remplissage de la colonne Delta_temps

In [70]:
df['Delta_temps'] = df.sort_values(['MMSI', 'BaseDateTime']).groupby('MMSI')['BaseDateTime'].diff().dt.total_seconds()

## Création des données d'entraînement:

Les navires n’envoient pas leurs signaux à des intervalles réguliers. Pour contourner ce problème, tous les signaux reçus entre 3 et 7 minutes après une position de référence sont considérés comme représentant la position du navire à +5 minutes. Il s’agit ici d’une fenêtre de tolérance de +/- 2 minutes.

La fonction ci dessous est **entièrement généré par IA**, les résultats semblent cohérents et je les utilises pour entrainer mes modèles.

In [71]:
from datetime import timedelta

def remplir_positions(df):
    df_result = []

    # Trier les données
    df = df.sort_values(["MMSI", "BaseDateTime"]).reset_index(drop=True)

    for mmsi, group in df.groupby("MMSI"):
        group = group.sort_values("BaseDateTime").reset_index(drop=True)
        for idx, row in group.iterrows():
            base_time = row["BaseDateTime"]

            # Fenêtres de temps en minutes
            t_windows = {
                '5': (base_time + timedelta(minutes=3), base_time + timedelta(minutes=7)),
                '10': (base_time + timedelta(minutes=8), base_time + timedelta(minutes=12)),
                '15': (base_time + timedelta(minutes=13), base_time + timedelta(minutes=17)),
            }

            positions = {}
            for key, (start, end) in t_windows.items():
                candidates = group[(group["BaseDateTime"] >= start) & (group["BaseDateTime"] <= end)]
                if not candidates.empty:
                    chosen = candidates.iloc[0]
                    positions[f"LAT_{key}"] = chosen["LAT"]
                    positions[f"LONG_{key}"] = chosen["LON"]
                else:
                    break  # Si une fenêtre n’a pas de donnée, on saute cette ligne
            else:
                df_result.append({**row, **positions})

    return pd.DataFrame(df_result)

# Appliquer sur la dataframe complète avant split
df_cleaned = remplir_positions(df)

# Affichage pour vérification
print(df_cleaned.head())
print(f"Nombre de lignes après nettoyage : {len(df_cleaned)}")

        id       MMSI        BaseDateTime       LAT       LON  SOG    COG  \
0  9625845  205776000 2023-05-25 00:31:27  25.95848 -97.37879  0.0   77.2   
1  9314767  205776000 2023-05-25 00:55:27  25.95842 -97.37880  0.0  162.9   
2  9353676  205776000 2023-05-25 01:04:27  25.95846 -97.37879  0.1  169.9   
3  9376837  205776000 2023-05-25 01:07:27  25.95845 -97.37879  0.2  185.8   
4  9422207  205776000 2023-05-25 01:13:27  25.95843 -97.37879  0.0  172.0   

   Heading VesselName         IMO  ... Draft  Cargo  TransceiverClass  \
0    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
1    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
2    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
3    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
4    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   

     LONG_5     LAT_5   LONG_10    LAT_10   LONG_15    LAT_15  Delta_temps  
0 -97.37882  25.95846

In [72]:
#récupére les MMSI uniques de df_cleaned
mmsi_uniques = df_cleaned['MMSI'].dropna().unique()

#mélangr
np.random.seed(42)
np.random.shuffle(mmsi_uniques)

#Répartition des MMSI en 3 bases
n_train = 97
n_test = 30
n_val = 22

mmsi_train = mmsi_uniques[:n_train]
mmsi_test = mmsi_uniques[n_train:n_train + n_test]
mmsi_val = mmsi_uniques[n_train + n_test:n_train + n_test + n_val]

#création des sous ensembles
df_train = df_cleaned[df_cleaned['MMSI'].isin(mmsi_train)]
df_test = df_cleaned[df_cleaned['MMSI'].isin(mmsi_test)]
df_val = df_cleaned[df_cleaned['MMSI'].isin(mmsi_val)]

#Vérif
print("Nouveaux ensembles avec df_cleaned :")
print("Train:", df_train['MMSI'].nunique())
print("Test :", df_test['MMSI'].nunique())
print("Val  :", df_val['MMSI'].nunique())


Nouveaux ensembles avec df_cleaned :
Train: 97
Test : 30
Val  : 22


## Séparation en bases d'apprentissage, test et validation par MMSI

Observation de début de chaque df

In [73]:
print(df_train.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_test.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_val.head(5))

        id       MMSI        BaseDateTime       LAT       LON  SOG    COG  \
0  9625845  205776000 2023-05-25 00:31:27  25.95848 -97.37879  0.0   77.2   
1  9314767  205776000 2023-05-25 00:55:27  25.95842 -97.37880  0.0  162.9   
2  9353676  205776000 2023-05-25 01:04:27  25.95846 -97.37879  0.1  169.9   
3  9376837  205776000 2023-05-25 01:07:27  25.95845 -97.37879  0.2  185.8   
4  9422207  205776000 2023-05-25 01:13:27  25.95843 -97.37879  0.0  172.0   

   Heading VesselName         IMO  ... Draft  Cargo  TransceiverClass  \
0    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
1    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
2    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
3    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
4    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   

     LONG_5     LAT_5   LONG_10    LAT_10   LONG_15    LAT_15  Delta_temps  
0 -97.37882  25.95846

In [74]:
df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)
print(df_train.head(10))


        id       MMSI        BaseDateTime       LAT       LON  SOG    COG  \
0  9625845  205776000 2023-05-25 00:31:27  25.95848 -97.37879  0.0   77.2   
1  9314767  205776000 2023-05-25 00:55:27  25.95842 -97.37880  0.0  162.9   
2  9353676  205776000 2023-05-25 01:04:27  25.95846 -97.37879  0.1  169.9   
3  9376837  205776000 2023-05-25 01:07:27  25.95845 -97.37879  0.2  185.8   
4  9422207  205776000 2023-05-25 01:13:27  25.95843 -97.37879  0.0  172.0   
5  9456484  205776000 2023-05-25 01:22:27  25.95847 -97.37880  0.1  151.4   
6  9477296  205776000 2023-05-25 01:28:27  25.95845 -97.37885  0.1  176.9   
7  9538373  205776000 2023-05-25 01:31:27  25.95841 -97.37876  0.1  158.2   
8  9524383  205776000 2023-05-25 01:37:27  25.95844 -97.37877  0.1  161.9   
9  9570341  205776000 2023-05-25 01:43:27  25.95845 -97.37878  0.1  160.8   

   Heading VesselName         IMO  ... Draft  Cargo  TransceiverClass  \
0    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
1    2

C:\Users\Paul\AppData\Local\Temp\ipykernel_20320\703263359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)


In [75]:
print(f"Taille de l'ensemble d'apprentissage : {len(df_train)}")
print(f"Taille de l'ensemble de test : {len(df_test)}")
print(f"Taille de l'ensemble de validation : {len(df_val)}")


Taille de l'ensemble d'apprentissage : 255966
Taille de l'ensemble de test : 73807
Taille de l'ensemble de validation : 53845


In [76]:
df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)
print(df_train.head())
# print(df_train.iloc[2950:3000])


        id       MMSI        BaseDateTime       LAT       LON  SOG    COG  \
0  9625845  205776000 2023-05-25 00:31:27  25.95848 -97.37879  0.0   77.2   
1  9314767  205776000 2023-05-25 00:55:27  25.95842 -97.37880  0.0  162.9   
2  9353676  205776000 2023-05-25 01:04:27  25.95846 -97.37879  0.1  169.9   
3  9376837  205776000 2023-05-25 01:07:27  25.95845 -97.37879  0.2  185.8   
4  9422207  205776000 2023-05-25 01:13:27  25.95843 -97.37879  0.0  172.0   

   Heading VesselName         IMO  ... Draft  Cargo  TransceiverClass  \
0    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
1    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
2    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
3    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
4    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   

     LONG_5     LAT_5   LONG_10    LAT_10   LONG_15    LAT_15  Delta_temps  
0 -97.37882  25.95846

C:\Users\Paul\AppData\Local\Temp\ipykernel_20320\1592785229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)


## Prédiction de LAT et LONG à horizon de 5 minutes

In [77]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

features = ['SOG', 'COG', 'Heading', 'LAT', 'LON']
X = df_train[features]
y_lat = df_train['LAT_5']
y_lon = df_train['LONG_5']

model_lat = RandomForestRegressor(
    n_estimators=20,    #20 arbres de décision et rofondeur max de 15
    max_depth = 15
)
model_lat.fit(X, y_lat)

model_lon = RandomForestRegressor(
    n_estimators=20,
    max_depth = 15
)
model_lon.fit(X, y_lon)


RandomForestRegressor(max_depth=15, n_estimators=20)

sauvegade du modèle

In [78]:
import joblib

#Sauvegarde
joblib.dump(model_lat, 'model5_lat.pkl')
joblib.dump(model_lon, 'model5_lon.pkl')

# model_lat = joblib.load('model_lat.pkl')
# model_lon = joblib.load('model_lon.pkl')

['model5_lon.pkl']

## calcul du RMSE (erreur quadratique moyenne) entre les valeurs prédites et les valeurs réelles

base de test

In [79]:
from sklearn.metrics import mean_squared_error
import numpy as np

X_test = df_test[features]
y_lat_test = df_test['LAT_5']
y_lon_test = df_test['LONG_5']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.0056846815697353765
RMSE LON: 0.00603254267883836


base de validation

In [80]:
from sklearn.metrics import mean_squared_error
import numpy as np

X_test = df_val[features]
y_lat_test = df_val['LAT_5']
y_lon_test = df_val['LONG_5']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.019997634875973093
RMSE LON: 0.005817717932602357


## Prédiction de LAT et LONG à horizon de 10 minutes

In [81]:
features = ['SOG', 'COG', 'Heading','LAT', 'LON']
X = df_train[features]
y_lat = df_train['LAT_10']
y_lon = df_train['LONG_10']

model_lat10 = RandomForestRegressor(
    n_estimators=20, #20 arbres de décision et rofondeur max de 15
    max_depth = 15)
model_lat10.fit(X, y_lat)

model_lon10 = RandomForestRegressor(
    n_estimators=20,
    max_depth = 15)
model_lon10.fit(X, y_lon)

RandomForestRegressor(max_depth=15, n_estimators=20)

In [82]:
import joblib

# Sauvegarde
joblib.dump(model_lat, 'model10_lat.pkl')
joblib.dump(model_lon, 'model10_lon.pkl')

# model_lat = joblib.load('model_lat.pkl')
# model_lon = joblib.load('model_lon.pkl')

['model10_lon.pkl']

calcul du RMSE (erreur quadratique moyenne) entre les valeurs prédites et les valeurs réelles

base de test

In [83]:
from sklearn.metrics import mean_squared_error
import numpy as np

X_test = df_test[features]
y_lat_test = df_test['LAT_10']
y_lon_test = df_test['LONG_10']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.009185108099090886
RMSE LON: 0.010378829150599966


base de validation

In [84]:
X_test = df_val[features]
y_lat_test = df_val['LAT_10']
y_lon_test = df_val['LONG_10']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.022013832497645408
RMSE LON: 0.012509372122377732


## Prédiction de LAT et LONG à horizon de 15 minutes

In [89]:
features = ['SOG', 'COG', 'Heading', 'LAT', 'LON']
X = df_train[features]
y_lat = df_train['LAT_15']
y_lon = df_train['LONG_15']

model_lat15 = RandomForestRegressor(
    n_estimators=20, #20 arbres de décision et rofondeur max de 15
    max_depth = 15
)
model_lat15.fit(X, y_lat)

model_lon15 = RandomForestRegressor(   
    n_estimators=20,
    max_depth = 15)
model_lon15.fit(X, y_lon)

RandomForestRegressor(max_depth=15, n_estimators=20)

In [90]:
# Sauvegarde
joblib.dump(model_lat, 'model15_lat.pkl')
joblib.dump(model_lon, 'model15_lon.pkl')

# model_lat = joblib.load('model_lat.pkl')
# model_lon = joblib.load('model_lon.pkl')

['model15_lon.pkl']

base de test

In [91]:
X_test = df_test[features]
y_lat_test = df_test['LAT_15']
y_lon_test = df_test['LONG_15']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.014375466483207129
RMSE LON: 0.016381204603927148


base de validation

In [92]:
X_test = df_val[features] 
y_lat_test = df_val['LAT_15']
y_lon_test = df_val['LONG_15']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.02617481266866374
RMSE LON: 0.020561092743758863


fin